<a href="https://colab.research.google.com/github/Ofiregev/Final_Project-FetalCns/blob/main/ofir's_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [60]:
from google.colab import drive
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os
import pandas as pd
import torch.nn.functional as F
from sklearn.model_selection import KFold
import numpy as np

# Mount Google Drive
drive.mount('/content/drive')

print("Part 1 has done")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Part 1 has done


In [61]:
# Part 2: Define CustomDataset Class and Read CSV File
class CustomDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.data_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.data_frame.iloc[idx, 0])
        image = Image.open(img_name)
        if self.transform:
            image = self.transform(image)

        label = self.data_frame.iloc[idx, 2]  # Assuming the third column is the label

        return image, label

In [62]:

# Determine the available hardware (CPU or GPU) and set the PyTorch device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cpu


In [63]:
# Read the list of images from the CSV file
train_csv = "/content/drive/MyDrive/FinalProject/training_set_pixel_size_and_HC.csv"
train_csv_df = pd.read_csv(train_csv)


In [64]:
train_csv_df.head()

,filename,pixel size(mm),head circumference (mm)
0,000_HC.png,0.069136,44.30
1,001_HC.png,0.089659,56.81
2,002_HC.png,0.062033,68.75
3,003_HC.png,0.091291,69.00
4,004_HC.png,0.061240,59.81


In [65]:
# Part 3: Define Transforms and Create Dataset
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor()
])
train_dataset = CustomDataset(csv_file="/content/drive/MyDrive/FinalProject/training_set_pixel_size_and_HC.csv",
                              root_dir="/content/drive/MyDrive/FinalProject/Dataset/training_set/training_set/",
                              transform=transform)

In [66]:
# Part 4: Create DataLoader
batch_size = 2
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [67]:
# Part 5: Define Model
class Model(nn.Module):
    def __init__(self, input_channels=1, input_size=64):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(input_channels, 64, 3, padding=1)
        self.conv2 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv3 = nn.Conv2d(128, 256, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        conv_output_size = input_size // 2 // 2 // 2  # Adjusted for additional pooling layer
        self.fc1 = nn.Linear(256 * conv_output_size * conv_output_size, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 1)
        self.dropout = nn.Dropout(0.5)  # Dropout probability of 0.5

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x


In [68]:
model = Model(input_channels=1, input_size=64)
model.to(device)

Model(
  (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=16384, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [69]:
# Part 6: Define Training Parameters and Optimizer
learning_rate = 0.001
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)


In [70]:
def train(model, train_loader, optimizer, criterion, epochs, device):
    model.to(device)
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0

        for inputs, labels in train_loader:
            inputs = inputs.float().to(device)
            labels = labels.float().unsqueeze(1).to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        epoch_loss = running_loss / len(train_loader)
        print(f"Epoch {epoch + 1}, Loss: {epoch_loss:.4f}")

In [71]:
def evaluate(model, val_loader, criterion, device):
    model.eval()
    val_losses = []

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs = inputs.float().to(device)
            labels = labels.float().unsqueeze(1).to(device)

            outputs = model(inputs)
            val_loss = criterion(outputs, labels)
            val_losses.append(val_loss.item())

    fold_avg_val_loss = np.mean(val_losses)
    return fold_avg_val_loss

In [73]:
k = 2
kf = KFold(n_splits=k, shuffle=True, random_state=42)
evaluation_metrics = []
epochs = 2

for fold_idx, (train_idx, val_idx) in enumerate(kf.split(train_dataset)):
    print(f"Fold {fold_idx + 1}")
    print("=========")

    train_data_fold = [train_dataset[i] for i in train_idx]
    val_data_fold = [train_dataset[i] for i in val_idx]

    train_loader_fold = DataLoader(train_data_fold, batch_size=batch_size, shuffle=True, num_workers=2)
    val_loader_fold = DataLoader(val_data_fold, batch_size=batch_size, num_workers=4)

    train(model, train_loader_fold, optimizer, criterion, epochs, device)
    fold_avg_val_loss = evaluate(model, val_loader_fold, criterion, device)
    evaluation_metrics.append(fold_avg_val_loss)

    print(f"Fold {fold_idx + 1} Validation Loss: {fold_avg_val_loss:.4f}")
    print("=========")

average_metric = np.mean(evaluation_metrics)
print("Average Evaluation Metric:", average_metric)

Fold 1


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Epoch 1, Loss: 4546.8128
Epoch 2, Loss: 4329.2132
Fold 1 Validation Loss: 2160.7534
Fold 2
Epoch 1, Loss: 3400.1795
Epoch 2, Loss: 3545.0643
Fold 2 Validation Loss: 2127.9158
Average Evaluation Metric: 2144.3345648937225


In [74]:
# Part 8: Save the trained model
model_path = "/content/drive/MyDrive/FinalProject/trained_model.pth"
torch.save(model.state_dict(), model_path)
print("Model saved successfully!")

Model saved successfully!


In [ ]:
# # Load the trained model
model_path = "/content/drive/MyDrive/FinalProject/trained_model.pth"
model = Model(input_channels=1, input_size=64)
model.load_state_dict(torch.load(model_path))
model.eval()

In [ ]:
# Function to predict head circumference from a new image
def predict_head_circumference(image_path, model):
    # Open and preprocess the new image
    transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor()
])
    image = Image.open(image_path)
    image_tensor = transform(image).unsqueeze(0)  # Add batch dimension

    # Make prediction
    with torch.no_grad():
        output = model(image_tensor.float())

    return output.item()  # Return the predicted head circumference as a scalar

In [77]:
# Path for the new image
new_image_path = "/content/drive/MyDrive/FinalProject/Dataset/training_set/training_set/001_HC.png"
# Call the function to predict head circumference from the new image
predicted_circumference = predict_head_circumference(new_image_path, model)
print("Predicted Head Circumference:", predicted_circumference)


Predicted Head Circumference: 83.84972381591797
